In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from pandas import Series

In [54]:
#Read Files
Uni_train = pd.read_csv("C:/Users/admn/Downloads/Unicorn/Train.csv")
Uni_test = pd.read_csv("C:/Users/admn/Downloads/Unicorn/Test.csv")

In [55]:
#Create Copy
Uni_train_copy = Uni_train.copy()
Uni_test_copy = Uni_test.copy()

In [56]:
Uni_train.columns

Index([u'ID', u'Datetime', u'Count'], dtype='object')

In [57]:
Uni_test.columns

Index([u'ID', u'Datetime'], dtype='object')

In [58]:
#Check Datatypes
Uni_train.dtypes

ID           int64
Datetime    object
Count        int64
dtype: object

In [59]:
Uni_test.dtypes

ID           int64
Datetime    object
dtype: object

In [60]:
Uni_train.shape

(18288, 3)

In [61]:
Uni_test.shape

(5112, 2)

In [62]:
#Convert Datatime into datatime format
Uni_train['Datetime'] = pd.to_datetime(Uni_train.Datetime,format='%d-%m-%Y %H:%M')
Uni_test['Datetime'] = pd.to_datetime(Uni_test.Datetime,format='%d-%m-%Y %H:%M')
#Uni_test_copy['Datetime'] = pd.to_datetime(Uni_test_copy.Datetime,format='%d-%m-%Y %H:%M')


#To Count No of Passengers on year,month,day and hour basis
for i in(Uni_train, Uni_test, Uni_train_copy, Uni_test_copy):
    i['year']=i.Datetime.dt.year
    i['month']=i.Datetime.dt.month
    i['day']=i.Datetime.dt.day
    i['hour']=i.Datetime.dt.hour
    
#To calculate weekdays also
Uni_train['day of week']=Uni_train['Datetime'].dt.dayofweek 
temp = Uni_train['Datetime']

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
#Assin 1 if weekend and O if not
def applyer(row):
    if row.dayofweek == 5 or row.dayofweek == 6:
        return 1
    else:
        return 0 
temp2 = Uni_train['Datetime'].apply(applyer) 
Uni_train['weekend']=temp2

In [ ]:
#Indexing
Uni_train.index = Uni_train['Datetime']
df = Uni_train.drop(['ID'], axis =1)
ts = df['Count']
plt.figure(figsize=(16,8)) 
plt.plot(ts, label='Passenger Count') 
plt.title('Time Series') 
plt.xlabel("Time(year-month)") 
plt.ylabel("Passenger count") 
plt.legend(loc='best')


In [ ]:
#Checking hupothesis
Uni_train.groupby('year')['Count'].mean().plot.bar()

In [ ]:
Uni_train.groupby('month')['Count'].mean().plot.bar()

In [ ]:
temp=Uni_train.groupby(['year', 'month'])['Count'].mean() 
temp.plot(figsize=(15,5), title= 'Passenger Count(Monthwise)', fontsize=14)

In [ ]:
Uni_train.groupby('day')['Count'].mean().plot.bar()

In [ ]:
Uni_train.groupby('hour')['Count'].mean().plot.bar()

In [ ]:
Uni_train.groupby('weekend')['Count'].mean().plot.bar()

In [ ]:
Uni_train.groupby('day of week')['Count'].mean().plot.bar()

In [ ]:
#Drop ID variable
Uni_train = Uni_train.drop(['ID'], axis = 1)

In [ ]:
#Aggregating hour with day, week and month to reduce noise
Uni_train.Timestamp = pd.to_datetime(Uni_train.Datetime,format='%d-%m-%Y %H:%M') 
Uni_train.index = Uni_train.Timestamp

# Hourly time series 
hourly = Uni_train.resample('H').mean()

# Converting to daily mean 
daily = Uni_train.resample('D').mean() 
# Converting to weekly mean 
weekly = Uni_train.resample('W').mean() 
# Converting to monthly mean 
monthly = Uni_train.resample('M').mean()

In [ ]:
fig, axs = plt.subplots(4,1) 
hourly.Count.plot(figsize=(15,8), title= 'Hourly', fontsize=14, ax=axs[0])
daily.Count.plot(figsize=(15,8), title= 'Daily', fontsize=14, ax=axs[1]) 
weekly.Count.plot(figsize=(15,8), title= 'Weekly', fontsize=14, ax=axs[2])
monthly.Count.plot(figsize=(15,8), title= 'Monthly', fontsize=14, ax=axs[3]) 

plt.show()

In [ ]:
#Test File
Uni_test.Timestamp = pd.to_datetime(Uni_test.Datetime,format='%d-%m-%Y %H:%M') 
Uni_test.index = Uni_test.Timestamp  

# Converting to daily mean 
Uni_test = Uni_test.resample('D').mean() 

#Train File
Uni_train.Timestamp = pd.to_datetime(Uni_train.Datetime,format='%d-%m-%Y %H:%M') 
Uni_train.index = Uni_train.Timestamp

# Converting to daily mean 
Uni_train = Uni_train.resample('D').mean()

In [ ]:
#Spliting the Train File
Uni_Train=Uni_train.ix['2012-08-25':'2014-06-24'] 
Uni_valid=Uni_train.ix['2014-06-25':'2014-09-25']

In [ ]:
Uni_Train.Count.plot(figsize=(15,8), title= 'Daily Ridership', fontsize=14, label='train')
Uni_valid.Count.plot(figsize=(15,8), title= 'Daily Ridership', fontsize=14, label='valid') 
plt.xlabel("Datetime") 
plt.ylabel("Passenger count") 
plt.legend(loc='best') 
plt.show()

In [ ]:
#Forecasting Using Naive Bayes
#Let’s make predictions using naive approach for the validation set.

dd= np.asarray(Uni_Train.Count) 
y_hat = Uni_valid.copy() 
y_hat['naive'] = dd[len(dd)-1] 
plt.figure(figsize=(12,8)) 
plt.plot(Uni_Train.index, Uni_Train['Count'], label='Train') 
plt.plot(Uni_valid.index, Uni_valid['Count'], label='Valid') 
plt.plot(y_hat.index,y_hat['naive'], label='Naive Forecast') 
plt.legend(loc='best') 
plt.title("Naive Forecast") 
plt.show()

In [ ]:
#We will now calculate RMSE to check the accuracy of our model on validation data set.

from sklearn.metrics import mean_squared_error 
from math import sqrt 
rms = sqrt(mean_squared_error(Uni_valid.Count, y_hat.naive)) 
print(rms)

In [ ]:
#Moving Average
#Here the predictions are made on the basis of the average of last few points instead of taking all the previously known values.

#ets try the rolling mean for last 10, 20, 50 days and visualize the results.

y_hat_avg = Uni_valid.copy()

#Calculating Average of last 10 Transactions
y_hat_avg['moving_avg_forecast'] = Uni_Train['Count'].rolling(10).mean().iloc[-1]
plt.figure(figsize=(15,5))
plt.plot(Uni_Train['Count'], label = 'Train')
plt.plot(Uni_valid['Count'],label = 'Valid')
plt.plot(y_hat_avg['moving_avg_forecast'], label = 'Moving Average Forecast using 10 observations')
plt.legend(loc='best')
plt.show()


#Calculating With last 20 transactions
y_hat_avg = Uni_valid.copy()
y_hat_avg['moving_avg_forecast'] = Uni_Train['Count'].rolling(20).mean().iloc[-1]
plt.figure(figsize=(15,5))
plt.plot(Uni_Train['Count'], label = 'Train')
plt.plot(Uni_valid['Count'],label = 'Valid')
plt.plot(y_hat_avg['moving_avg_forecast'], label = 'Moving Average Forecast using 20 observations')
plt.legend(loc='best')
plt.show()


#Calculating with last 50 transactions
y_hat_avg = Uni_valid.copy()
y_hat_avg['moving_avg_forecast'] = Uni_Train['Count'].rolling(50).mean().iloc[-1]
plt.figure(figsize=(15,5))
plt.plot(Uni_Train['Count'], label = 'Train')
plt.plot(Uni_valid['Count'],label = 'Valid')
plt.plot(y_hat_avg['moving_avg_forecast'], label = 'Moving Average Forecast using 50 observations')
plt.legend(loc='best')
plt.show()

In [ ]:
rms = sqrt(mean_squared_error(Uni_valid.Count, y_hat_avg.moving_avg_forecast)) 
print(rms)

In [ ]:
#Simple Exponential Smoothing
#Here the predictions are made by assigning larger weight to the recent values and lesser weight to the old values.

from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt 
y_hat_avg = Uni_valid.copy() 
fit2 = SimpleExpSmoothing(np.asarray(Uni_Train['Count'])).fit(smoothing_level=0.6,optimized=False)
y_hat_avg['SES'] = fit2.forecast(len(Uni_valid)) 
plt.figure(figsize=(16,8)) 
plt.plot(Uni_Train['Count'], label='Train') 
plt.plot(Uni_valid['Count'], label='Valid') 
plt.plot(y_hat_avg['SES'], label='SES') 
plt.legend(loc='best') 
plt.show()

In [ ]:
rms = sqrt(mean_squared_error(Uni_valid.Count, y_hat_avg.SES)) 
print(rms)

In [ ]:
#Holt’s Linear Trend Model
import statsmodels.api as sm 
sm.tsa.seasonal_decompose(Uni_Train.Count).plot() 
result = sm.tsa.stattools.adfuller(Uni_train.Count) 
plt.show()

In [ ]:
#An increasing trend can be seen in the dataset, so now we will make a model based on the trend.

y_hat_avg = Uni_valid.copy() 
fit1 = Holt(np.asarray(Uni_Train['Count'])).fit(smoothing_level = 0.3,smoothing_slope = 0.1) 
y_hat_avg['Holt_linear'] = fit1.forecast(len(Uni_valid)) 
plt.figure(figsize=(16,8)) 
plt.plot(Uni_Train['Count'], label='Train') 
plt.plot(Uni_valid['Count'], label='Valid') 
plt.plot(y_hat_avg['Holt_linear'], label='Holt_linear') 
plt.legend(loc='best') 
plt.show()

In [ ]:
rms = sqrt(mean_squared_error(Uni_valid.Count, y_hat_avg.Holt_linear)) 
print(rms)

In [ ]:
#3) Holt’s Linear Trend Model on daily time series
submission=pd.read_csv("C:/Users/admn/Downloads/Unicorn/Submission.csv")


In [ ]:
predict=fit1.forecast(len(Uni_test))

In [ ]:
Uni_test['prediction']=predict

In [ ]:
# Calculating the hourly ratio of count 
Uni_train_copy['ratio']=Uni_train_copy['Count']/Uni_train_copy['Count'].sum() 

In [ ]:
# Grouping the hourly ratio 
temp=Uni_train_copy.groupby(['hour'])['ratio'].sum()

In [ ]:
# Groupby to csv format 
pd.DataFrame(temp, columns=['hour','ratio']).to_csv('GROUPby.csv')

In [ ]:
temp2=pd.read_csv("GROUPby.csv") 
temp2=temp2.drop('hour.1',1)

In [ ]:
# Merge Test and test_original on day, month and year 
merge=pd.merge(Uni_test, Uni_test_copy, on=('day','month', 'year'), how='left') 
merge['hour'] = merge['hour_y'] 
merge=merge.drop(['year', 'month', 'Datetime','hour_x','hour_y'], axis=1)

In [ ]:
# Predicting by merging merge and temp2 
prediction=pd.merge(merge, temp2, on='hour', how='left')

In [ ]:
# Converting the ratio to the original scale 
prediction['Count']=prediction['prediction']*prediction['ratio']*24 
prediction['ID']=prediction['ID_y']

In [ ]:
#Let’s drop all other features from the submission file and keep ID and Count only.

submission=prediction.drop(['ID_x', 'day', 'ID_y','prediction','hour', 'ratio'],axis=1)

In [ ]:
# Converting the final submission to csv format 
pd.DataFrame(submission, columns=['ID','Count']).to_csv('Holt linear.csv')

In [ ]:
#########################################Holt winter’s model on daily time series

In [ ]:
y_hat_avg = Uni_valid.copy() 
fit1 = ExponentialSmoothing(np.asarray(Uni_Train['Count']) ,seasonal_periods=7 ,trend='add', seasonal='add',).fit() 
y_hat_avg['Holt_Winter'] = fit1.forecast(len(Uni_valid)) 
plt.figure(figsize=(16,8)) 
plt.plot(Uni_Train['Count'], label='Train') 
plt.plot(Uni_valid['Count'], label='Valid') 
plt.plot(y_hat_avg['Holt_Winter'], label='Holt_Winter') 
plt.legend(loc='best') 
plt.show()

In [ ]:
rms = sqrt(mean_squared_error(Uni_valid.Count, y_hat_avg.Holt_Winter))
print(rms)

In [ ]:
predict=fit1.forecast(len(Uni_test))

In [ ]:
#Now we will convert these daily passenger count into hourly passenger count using the same approach which we followed above.

Uni_test['prediction']=predict

In [ ]:
# Merge Test and test_original on day, month and year 
merge=pd.merge(Uni_test, Uni_test_copy, on=('Day','month', 'year'), how='left') 
merge['hour']=merge['hour_y'] 
merge=merge.drop(['year', 'month', 'Datetime','hour_x','hour_y'], axis=1) 

In [ ]:
# Predicting by merging merge and temp2 
prediction=pd.merge(merge, temp2, on='hour', how='left') 

In [ ]:
#Converting the ratio to the original scale 
prediction['Count']=prediction['prediction']*prediction['ratio']*24
#Let’s drop all features other than ID and Count

prediction['ID']=prediction['ID_y'] 
submission=prediction.drop(['day','hour','ratio','prediction', 'ID_x', 'ID_y'],axis=1) 

In [ ]:
# Converting the final submission to csv format
pd.DataFrame(submission, columns=['ID','Count']).to_csv('Holt winters.csv')

In [ ]:
#ARIMA Model

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determining rolling statistics
    rolmean = timeseries.rolling(window = 24).mean()
    rolSTD = timeseries.rolling(window = 24).std()
    
    #Plot Rolling statistics
    orig = plt.plot(timeseries, color = 'blue', label = 'Original')
    mean = plt.plot(rolmean, color = 'Red', label = 'Mean')
    std = plt.plot(rolSTD, color = 'Black', label = 'Std')
    plt.legend(loc = 'best')
    plt.title("Rolling Mean and STD deviation")
    plt.show(block=False)   
    
    
    #Dickey Fuller Test
    print("Results of Dickey Fuller test")
    dftest = adfuller(timeseries, autolag='AIC')
    dfop = pd.Series(dftest[0:4], index=['Test-statistics', 'pvalue', '#Lags used', 'No of observations'])
    for key,value in dftest[4].items():
        dfop['critical value (%s)'%key] = value
    print(dfop)

In [ ]:
from matplotlib.pylab import rcParams 
rcParams['figure.figsize'] = 20,10
test_stationarity(Uni_train_copy['Count'])

In [ ]:
#Removing Trend
Train_log = np.log(Uni_Train['Count']) 
valid_log = np.log(Uni_valid['Count'])
moving_avg = Train_log.rolling(24).mean()
plt.plot(Train_log) 
plt.plot(moving_avg, color = 'red') 
plt.show()

In [ ]:
#Removing this Trend
train_log_moving_avg_diff = Train_log - moving_avg

In [ ]:
#Since we took the average of 24 values, rolling mean is not defined for the first 23 values. So let’s drop those null values.

train_log_moving_avg_diff.dropna(inplace = True) 
test_stationarity(train_log_moving_avg_diff)

In [ ]:
#We can see that the Test Statistic is very smaller as compared to the Critical Value. So, we can be confident that the trend is almost removed.

#Let’s now stabilize the mean of the time series which is also a requirement for a stationary time series.

#Differencing can help to make the series stable and eliminate the trend.
train_log_diff = Train_log - Train_log.shift(1) 
test_stationarity(train_log_diff.dropna())

In [ ]:
#Removing Seasonality
from statsmodels.tsa.seasonal import seasonal_decompose 
decomposition = seasonal_decompose(pd.DataFrame(Train_log).Count.values, freq = 24) 

trend = decomposition.trend 
seasonal = decomposition.seasonal 
residual = decomposition.resid 

plt.subplot(411) 
plt.plot(Train_log, label='Original') 
plt.legend(loc='best') 
plt.subplot(412) 
plt.plot(trend, label='Trend') 
plt.legend(loc='best') 
plt.subplot(413) 
plt.plot(seasonal,label='Seasonality') 
plt.legend(loc='best') 
plt.subplot(414) 
plt.plot(residual, label='Residuals') 
plt.legend(loc='best') 
plt.tight_layout() 
plt.show()


In [ ]:
#Checking Stationarity in Residuals
train_log_decompose = pd.DataFrame(residual) 
train_log_decompose['date'] = Train_log.index 
train_log_decompose.set_index('date', inplace = True) 
train_log_decompose.dropna(inplace=True) 
test_stationarity(train_log_decompose[0])


In [ ]:
#Forecasting the time series using ARIMA

#First of all we will fit the ARIMA model on our time series for that we have to find the optimized values for the p,d,q parameters.

#To find the optimized values of these parameters, we will use ACF(Autocorrelation Function) and PACF(Partial Autocorrelation Function) graph.

#PACF measures the correlation between the TimeSeries with a lagged version of itself but after eliminating the variations already explained by the intervening comparisons.

from statsmodels.tsa.stattools import acf, pacf 
lag_acf = acf(train_log_diff.dropna(), nlags=25) 
lag_pacf = pacf(train_log_diff.dropna(), nlags=25, method='ols')

In [ ]:
plt.plot(lag_acf) 
plt.axhline(y=0,linestyle='--',color='gray') 
plt.axhline(y=-1.96/np.sqrt(len(train_log_diff.dropna())),linestyle='--',color='gray') 
plt.axhline(y=1.96/np.sqrt(len(train_log_diff.dropna())),linestyle='--',color='gray') 
plt.title('Autocorrelation Function') 
plt.show() 
plt.plot(lag_pacf) 
plt.axhline(y=0,linestyle='--',color='gray') 
plt.axhline(y=-1.96/np.sqrt(len(train_log_diff.dropna())),linestyle='--',color='gray') 
plt.axhline(y=1.96/np.sqrt(len(train_log_diff.dropna())),linestyle='--',color='gray') 
plt.title('Partial Autocorrelation Function') 
plt.show()


In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(Train_log, order=(2, 1, 0))  # here the q value is zero since it is just the AR model 
results_AR = model.fit(disp=-1)  
plt.plot(train_log_diff.dropna(), label='original') 
plt.plot(results_AR.fittedvalues, color='red', label='predictions') 
plt.legend(loc='best') 
plt.show()

In [ ]:
#Lets plot the validation curve for AR model.

#We have to change the scale of the model to the original scale.

#First step would be to store the predicted results as a separate series and observe it.

AR_predict=results_AR.predict(start="2014-06-25", end="2014-09-25") 
AR_predict=AR_predict.cumsum().shift().fillna(0) 
AR_predict1=pd.Series(np.ones(Uni_valid.shape[0]) * np.log(Uni_valid['Count'])[0], index = Uni_valid.index) 
AR_predict1=AR_predict1.add(AR_predict,fill_value=0) 
AR_predict = np.exp(AR_predict1)
plt.plot(Uni_valid['Count'], label = "Valid") 
plt.plot(AR_predict, color = 'red', label = "Predict") 
plt.legend(loc= 'best') 
plt.title('RMSE: %.4f'% (np.sqrt(np.dot(AR_predict, Uni_valid['Count']))/Uni_valid.shape[0])) 
plt.show()

In [ ]:
#Ma Model
#The moving-average model specifies that the output variable depends linearly on the current and various past values of a stochastic (imperfectly predictable) term.

model = ARIMA(Train_log, order=(0, 1, 2))  # here the p value is zero since it is just the MA model 
results_MA = model.fit(disp=-1)  
plt.plot(train_log_diff.dropna(), label='original') 
plt.plot(results_MA.fittedvalues, color='red', label='prediction') 
plt.legend(loc='best') 
plt.show()


In [ ]:
MA_predict=results_MA.predict(start="2014-06-25", end="2014-09-25") 
MA_predict=MA_predict.cumsum().shift().fillna(0) 
MA_predict1=pd.Series(np.ones(Uni_valid.shape[0]) * np.log(Uni_valid['Count'])[0], index = Uni_valid.index) 
MA_predict1=MA_predict1.add(MA_predict,fill_value=0) 
MA_predict = np.exp(MA_predict1)
plt.plot(Uni_valid['Count'], label = "Valid") 
plt.plot(MA_predict, color = 'red', label = "Predict") 
plt.legend(loc= 'best') 
plt.title('RMSE: %.4f'% (np.sqrt(np.dot(MA_predict, Uni_valid['Count']))/Uni_valid.shape[0])) 
plt.show()


In [ ]:
#Combining_Model
model = ARIMA(Train_log, order=(2, 1, 2))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(train_log_diff.dropna(),  label='original') 
plt.plot(results_ARIMA.fittedvalues, color='red', label='predicted') 
plt.legend(loc='best') 
plt.show()


In [ ]:
#Let’s define a function which can be used to change the scale of the model to the original scale.

def check_prediction_diff(predict_diff, given_set):
    predict_diff= predict_diff.cumsum().shift().fillna(0)
    predict_base = pd.Series(np.ones(given_set.shape[0]) * np.log(given_set['Count'])[0], index = given_set.index)
    predict_log = predict_base.add(predict_diff,fill_value=0)
    predict = np.exp(predict_log)

    plt.plot(given_set['Count'], label = "Given set")
    plt.plot(predict, color = 'red', label = "Predict")
    plt.legend(loc= 'best')
    plt.title('RMSE: %.4f'% (np.sqrt(np.dot(predict, given_set['Count']))/given_set.shape[0]))
    plt.show()

def check_prediction_log(predict_log, given_set):
    predict = np.exp(predict_log)
 
    plt.plot(given_set['Count'], label = "Given set")
    plt.plot(predict, color = 'red', label = "Predict")
    plt.legend(loc= 'best')
    plt.title('RMSE: %.4f'% (np.sqrt(np.dot(predict, given_set['Count']))/given_set.shape[0]))
    plt.show()

In [ ]:
#Let’s predict the values for validation set.

ARIMA_predict_diff=results_ARIMA.predict(start="2014-06-25", end="2014-09-25")
check_prediction_diff(ARIMA_predict_diff, Uni_valid)
